# Used Cars Price Prediction in Egypt Market

## Business Understanding:
### In this project, we will work with used cars dataset that has been scrapped with BeautufulSoup library from an Egyptian used cars website . This site has a lot of used cars ads in Egypt. So the project will focus on Egyptian used cars market. The target is to predict the car price range for either the buyer or the seller so they both can benefit from this service.

## Data Understanding:

### This dataset has 8 columns ( Make: manufacturer of vehicle, Model: model of vehicle, year: used since this year, CC: cubic centimeter or engine displacement which is a feature of the engine power, distance: the kilometers that the car traveled, transmission: manual or automatic gearbox, color: the color of the car, price: the car price )
### note: all cars are gas fuel Sedan cars type. The prices are by EGP.


In [124]:
import csv
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
from datasist.structdata import detect_outliers

In [181]:
pd.set_option("display.max_rows", None)

In [145]:
element = 'https://eg.website.com/en/car/search?make=&model=0&city=0&body=1&transmission=0&fuel=&priceMin=35000&priceMax=350000&kmMin=&kmMax=&dateMin=1990&dateMax=2020&color=&accountMin=&accountMax=&installmentMin=&installmentMax=&page='

In [147]:
links=[]

for i in range(407):
    response = requests.get(element+str(i))
    src = BeautifulSoup(response.content,'html.parser')
    cars = src.find_all('div' , {'class':"newCarListUnit_header"})
    for i in cars:
        link = i.find('a')['href']
        links.append(link)
    
    
urls=[]
for link in links:
    url='https://eg.website.com' + link
    urls.append(url)

    
len(urls)

12210

In [149]:
with open('data.csv','w', encoding='utf-8',newline='') as myfile:
        c_names=['Make','Model','year', 'CC','distance','transmission','color','price']
        writer = csv.DictWriter(myfile, fieldnames=c_names)
        writer.writeheader()
        for url in urls:
            try:
                get_url = requests.get(url)
                soup = BeautifulSoup(get_url.text , 'html.parser')
                dets = soup.find('div' , {'class':"DescDataRow"} ).find_all('div')
                price = soup.find_all('span' , {'class':"usedUnitCarPrice"})[0].get_text().strip()
                for item in dets:
                    info = item.get_text().strip().split('\n')
                    if info[0] == 'Make':
                        make = info[-1]
                    elif info[0] == 'Model':
                        model0 = info[-1]
                    elif info[0] == 'Used since':
                        year = info[-1]
                    elif info[0] == 'Km':
                        distance = info[-1]
                    elif info[0] == 'CC':
                        cc = info[-1]
                    elif info[0] == 'Transmission':
                        trans = info[-1]
                    elif info[0] == 'Color':
                        color = info[-1]

                writer.writerow({'Make': make ,'Model': model0 ,'year': year, 'CC': cc , 'distance': distance,'transmission': trans,'color': color,'price': price})       
            except:
                pass

In [213]:
df = pd.read_csv('data.csv')
df.head(10)

,Unnamed: 0,Make,Model,year,CC,distance,transmission,color,price
0,0,Chevrolet,Aveo,2012,1500,99000,manual,Silver,127000
1,1,Volkswagen,Jetta,2008,1500,200000,automatic,Silver,179000
2,2,Opel,Astra,2020,1500,57000,automatic,Silver,325000
3,3,Skoda,Octavia A5,2007,1600,150000,automatic,Gray,163000
4,4,Hyundai,Accent RB,2015,1600,135000,automatic,Black,190000
5,5,Hyundai,Accent RB,2011,1600,125000,automatic,Green,175000
6,6,Geely,Emgrand 7,2014,1600,91000,manual,Light grey,95000
7,7,Nissan,Sentra,2016,1600,103000,automatic,Gray,260000
8,8,Kia,Rio,2013,1400,165000,automatic,Silver,205000
9,9,Nissan,Sunny,2011,1600,150000,manual,Purple,140000


In [203]:
df.drop('Unnamed: 0' , axis=1, inplace=True)

### Distniguesh features

In [204]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6471 entries, 0 to 6470
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Make          6471 non-null   object
 1   Model         6471 non-null   object
 2   year          6471 non-null   int64 
 3   CC            6471 non-null   int64 
 4   distance      6471 non-null   int64 
 5   transmission  6471 non-null   object
 6   color         6471 non-null   object
 7   price         6471 non-null   int64 
dtypes: int64(4), object(4)
memory usage: 404.6+ KB


# Data cleaning:

### Problems:

### Numerical;
#### CC, price and distanc are not numerical


In [152]:
def cc(row):
    try:
        return int(row[:-3])
    except:
        return np.nan

In [153]:
df['CC'] = df['CC'].apply(cc)


In [154]:
def distance(row):
    try:
        return int(row[:-3].replace(',',""))
    except:
        return np.nan
    

In [155]:
df['distance'] = df['distance'].apply(distance)

In [156]:
def price(row):
    try:
        return int(row[:-4].replace(',',""))
    except:
        return np.nan
    
    
df['price'] = df['price'].apply(price)

### Data cleaning : Categorical values handling

In [157]:
df.head()

,Make,Model,year,CC,distance,transmission,color,price
0,Hyundai,Accent,2011,3500,109000,automatic,Silver,160000
1,Chevrolet,Aveo,2012,1500,99000,manual,Silver,127000
2,Volkswagen,Jetta,2008,1500,200000,automatic,Silver,179000
3,Opel,Astra,2020,1500,57000,automatic,Silver,325000
4,Skoda,Octavia A5,2007,1600,150000,automatic,Gray,163000


In [158]:
def no_space(row):
    try:
        return row.strip()
    except:
        return np.nan

In [159]:
df['Make'] = df['Make'].apply(no_space)
df['color'] = df['color'].apply(no_space)
df['Model'] = df['Model'].apply(no_space)

In [205]:
df['Make'].value_counts()

Hyundai       1024
Chevrolet      857
Daewoo         494
Nissan         471
Renault        460
Kia            437
Mitsubishi     362
Skoda          232
Opel           225
Speranza       208
Byd            206
Geely          201
Toyota         194
Fiat           192
Lada           125
Peugeot        112
Volkswagen      83
Chery           78
Proton          77
Seat            53
BMW             51
Mazda           47
Honda           43
Ford            42
MG              36
Citroën         30
Brilliance      26
Mercedes        22
Volvo           12
Subaru          12
Senova          11
Saipa            9
Haima            7
Suzuki           5
Great Wall       3
Soueast          3
Ds               2
Emgrand          2
Faw              2
Zotye            2
Chrysler         2
Jac              2
Changan          2
Audi             2
Pontiac          1
DFM              1
Chana            1
Daihatsu         1
Lifan            1
Name: Make, dtype: int64

# Handling Outliers

In [206]:
inx = detect_outliers(df, 0, ['CC'])
df.loc[inx].sort_values('CC')


,Make,Model,year,CC,distance,transmission,color,price


In [207]:
inx = detect_outliers(df, 0, ['price'])
df.loc[inx].sort_values('price')


,Make,Model,year,CC,distance,transmission,color,price
1622,MG,6,2020,1600,25000,automatic,Black,320000
4768,Mercedes,C 180,2009,1600,80000,automatic,Light grey,320000
2308,MG,6,2020,1500,84000,automatic,Black,320000
1018,Mercedes,S 320,2001,1600,244000,automatic,Gold,320000
4767,Toyota,Corolla,2020,1600,35000,automatic,White,320000
1150,Kia,K8,2019,1600,1000,automatic,Light grey,320000
1388,Toyota,Corolla,2015,1600,95000,automatic,Silver,320000
1457,Mazda,3 || Mazda 3,2018,1600,120000,automatic,Dark blue,320000
4582,Toyota,Corolla,2019,1600,93000,automatic,Silver,320000
4576,MG,6,2020,1500,38000,automatic,Gray,320000


In [208]:
len(inx)

108

In [198]:
df.drop(df[  df['distance'] == 0].index, axis=0,inplace=True)

In [209]:
df[  df['distance'] == 0].index


Int64Index([], dtype='int64')

# Missing Values

In [210]:
df.isna().sum()

Make            0
Model           0
year            0
CC              0
distance        0
transmission    0
color           0
price           0
dtype: int64

In [ ]:
df.dropna(axis=0, inplace=True)

## Handling Duplicated values

In [211]:
df[ df.duplicated() ]

,Make,Model,year,CC,distance,transmission,color,price


In [168]:
df.drop_duplicates(inplace=True)

In [169]:
inx = df[ df['price'] < 20000 ].index
inx

Int64Index([], dtype='int64')

In [64]:
df.drop(inx , axis=0, inplace=True)

In [180]:
df.shape

(9197, 8)

## Saving dataset

In [212]:
df.to_csv('data.csv')